In [222]:
from requests import get

from bs4 import BeautifulSoup
import pandas as pd
import time
import status

from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from translate import Translator
from langdetect import detect
import math 

stemmer = SnowballStemmer('english')

In [19]:

url = 'https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag='
pg = 1
tm = 3
df_info = pd.DataFrame(columns=['url','price','locali','superficie','bagni','piano'])
def scrapInfo(df_info, maxEntries):
    global pg, tm
    i = len(df_info.index)
    while i <= maxEntries:
        
        time.sleep(tm)
        response = get(url+str(pg))
        html_soup = BeautifulSoup(response.text, 'html.parser')
        #print (response)
        item_containers = html_soup.find_all('div', class_ = 'listing-item_body--content')
        #print(len(item_containers))
        for item in item_containers:
            listing = []
            listing.append(item.a['href'])
            #print(listing)    
            for d in item.ul.find_all('li', class_ = 'lif__item'):
                try:
                    
                    if 'lif__pricing' in d['class']:
                        listing.append(d.text.strip().split(' ')[1])
                        next
                    else:
                        if d.find('div', class_='lif__text').text == 'locali':
                            listing.append(d.span.text)
                        if d.find('div', class_='lif__text').text == 'superficie':
                            listing.append(d.span.text)
                        if d.find('div', class_='lif__text').text == 'bagni':
                            listing.append(d.span.text)
                        if d.find('div', class_='lif__text').text == 'piano':
                            listing.append(d.abbr.text.strip())
                except:
                    next
            #print(len(listing))
            if len(listing) == 6:
                df_info.loc[len(df_info.index)+1] = listing
                i +=1
                #tm += 0.2
        pg+=1
    return(df_info)

In [20]:
df_info = scrapInfo(df_info, 10000)

In [1]:
df_info.head()

NameError: name 'df_info' is not defined

In [16]:
pg

18

In [29]:
for i,row in df_info[0:3].iterrows():
    print(i)

1
2
3


In [120]:
#df_desc = pd.DataFrame(columns=['id','title','descrition'])
#html_zuppa = BeautifulSoup(response.text, 'html.parser')
#url_pre = 'https://www.immobiliare.it/nuove_costruzioni/Roma/'
#url_suf = '-immobile.html'
#pg_desc = 0
tm = 2
failedToRetrieve = []
def scrapDetails(df_desc, n):
    global pg_desc, tm, df_info

    for index, row in df_info[pg_desc:pg_desc+n].iterrows():
        listing = []
        listing.append(index)
        url_ = row['url']
        response = get(url_)
        #print(url_)
        if response.status_code == 200:
            try:
                html_zuppa = BeautifulSoup(response.text, 'html.parser')
                listing.append(html_zuppa.find('h1', class_='raleway title-detail').text.strip())
                listing.append(html_zuppa.find('div', class_='description-text').div.text.strip())
                df_desc.loc[len(df_desc.index)+1] = listing
                pg_desc += 1
                time.sleep(tm)
            except:
                failedToRetrieve.append(index)
                df_desc.loc[len(df_desc.index)+1] = [index,None,None]
                pg_desc += 1
                
        else:
            failedToRetrieve.append(index)
            df_desc.loc[len(df_desc.index)+1] = [index,None,None]
            pg_desc += 1

    return(df_desc)

In [156]:
print(*failedToRetrieve)

5002 5057 5362 5635 5643 5809 6322 6797 7039 7110 7132 7313 7314 7315 7316 7356 7374 7447 7463 7596 7597 7619 7744 7883 7884 7908 7922 7956 8215 8276 8360 8456 8554 8633 8665 8709 8838 9017 9051 9166 9292 9370 9479 9528 9555 9655 9731 9733 9734 9935


In [92]:
#for i in range(2100, 2039, -1):
#    df_desc.loc[i] = df_desc.loc[i-1]

In [75]:
df_desc = scrapDetails(df_desc, 100)

In [149]:
df_desc = scrapDetails(df_desc, 1000)


In [161]:
#clean datasets, byt deleting from info the entries which failed to retrieve their description
ds_info = df_info.drop(df_info.index[failedToRetrieve])


In [166]:
ds_desc = df_desc.drop(df_desc.index[failedToRetrieve])

In [170]:
ds_info.index = range(len(ds_info.index))
ds_desc.index = range(len(ds_desc.index))

In [150]:
#pg_desc

10011

In [168]:
ds_desc.describe()

,id,title,descrition
count,9961.0,9908,9908
unique,9960.0,8259,9815
top,2040.0,"Trilocale via Massa di San Giuliano 270, Roma",tel 3468137428 06 90288424 www.domusgallaplaci...
freq,2.0,14,3


In [169]:
ds_info.describe()

,url,price,locali,superficie,bagni,piano
count,9961,9961,9961,9961,9961,9961
unique,9935,868,6,334,4,15
top,https://www.immobiliare.it/67435127-Vendita-Tr...,199.000,3,100,1,1
freq,2,166,3661,480,5151,2210


In [56]:
df_desc[df_desc.duplicated(subset=['title', 'descrition'], keep= False)]

,id,title,descrition
333,333,"Bilocale via Torri di Castel di Leva 12, Roma",Rif: VRE - Via Torri di Castel di Leva. Adiace...
533,533,"Bilocale via Torri di Castel di Leva 12, Roma",Rif: VRE - Via Torri di Castel di Leva. Adiace...
848,848,"Trilocale via massimo troisi, 12, Roma","A ridosso di via di Malafede, Località Giardin..."
849,849,"Trilocale via massimo troisi, 12, Roma","A ridosso di via di Malafede, Località Giardin..."


In [108]:
df_info.loc[3919].url

'https://www.immobiliare.it/70370910-Vendita-Quadrilocale-via-Silvestri-Roma.html'

In [165]:
df_desc.describe()

,id,title,descrition
count,10011.0,9952,9952
unique,10010.0,8292,9858
top,2040.0,"Trilocale via Massa di San Giuliano 270, Roma",Rif: Appartamento di Nuova Costruzione In Clas...
freq,2.0,14,3


In [178]:
ds_desc.to_csv('ds_desc.csv', header = True)

In [177]:
ds_info.to_csv('ds_info.csv', header = True)

In [223]:
def cleanData(txt, lang='italian'):
    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = set(stopwords.words(lang))

    # get words lowercased
    t0 = str(txt).lower()
    # remove puctuations
    t1 = tokenizer.tokenize(t0)
    # reomve stop words
    t2 =[]
    t2 = [t1[i] for i in range(0,len(t1)) if t1[i] not in stop_words]

    # stemm words
    t3 = [stemmer.stem(t2[i]) for i in range(0, len(t2))]

    # remove nummbers and strings starting with numbers
    t4 = [t3[i] for i in range(0, len(t3)) if t3[i][0].isdigit()==False]
        
    return(t4)
    
def txtWords(txt):
    return(cleanData(txt))
def getTF(txt, word):
    cTxt = cleanData(txt)
    if word not in cTxt: return (0)
    return(cTxt.count(word)/len(cTxt))
    

In [264]:
pause = 0
class immobiliare:
    def __init__(self, dataset):
        self.data = dataset
        self.size = len(self.data)
        #vocabulary = self.getVoc()
        self._index = self.index()
        self._indexTFIDF = []
        global pause
        
    #def getVoc(self):
        
        
    def index(self):
        _index = {}
        for index, row in self.data.iterrows():
            for w in (list(set(txtWords(row.title+row.descrition)))):
                if w in _index.keys(): _index[w].append(index)
                else: _index[w] = [index]
        return(_index)
               
    def getIDF(self, word):
               return(math.log(self.size/len(self._index[word])))
               
    def getTFIDF(self, txt, word):
               if word not in cleanData(txt): return (0)
               return(getTF(txt, word) * self.getIDF(word))
            
# Consider using pyspark for this step!
    def buildIndexTFIDF(self, quanto, pause=0):
        tmpRow = []
        _indexTFIDF = []
        for w in list(self._index.keys())[pause:pause+quanto]:
            tmpRow.append([self.getTFIDF(ann.title+ann.descrition, w) for index, ann in self.data.iterrows()])
            _indexTFIDF.append(tmpRow)
            pause +=1
            self._indexTFIDF = _indexTFIDF
            return(_indexTFIDF)

In [258]:
immData = immobiliare(ds_desc)

In [ ]:
immData.buildIndexTFIDF(10, pause)

In [ ]:
immData._indexTFIDF

In [256]:
for w in list(immData._index.keys())[0:5]:
    print (w)

rifinitissimo
orari
giovedi
venrdi
domenica
